# Tidy Data

* Introduction
* Tidy data
    * Spreading and gathering
    * Separating and uniting

## Introduction
Once we bring in the data from an outside source there is no guarantee that it is in a standard format that we are used to. This consistent, clean, and standard format is called **tidy data**. What exactly this tidy format is might not be something that everyone agrees on, but there are aspects of it that are agreeable. Here we will go over some of these methods, will introduce tools provided by **pandas**, and by using them will show examples on how to tidy our data and how to join multiple tidy datasets to get the appropriate data we need in our reporting and analysis.

It is typical for a data science project to spend significant amount of time in tidying, transforming and manipulating the data. This is a crucial part of the project that is sometimes overlooked; the output only can get as good as the input, spending some upfront time on the data usually pays off in the end.

## Tidy data
You can represent the same underlying data in multiple ways. The example below shows the same data organized in four different ways. Each dataset shows the same values of four variables `country`, `year`, `population`, and `cases`:


In [1]:
import pandas as pd

table1  = pd.read_csv('https://raw.githubusercontent.com/soltaniehha/Intro-to-Data-Analytics/main/data/table1.csv')
table2  = pd.read_csv('https://raw.githubusercontent.com/soltaniehha/Intro-to-Data-Analytics/main/data/table2.csv')
table3  = pd.read_csv('https://raw.githubusercontent.com/soltaniehha/Intro-to-Data-Analytics/main/data/table3.csv')
table4a = pd.read_csv('https://raw.githubusercontent.com/soltaniehha/Intro-to-Data-Analytics/main/data/table4a.csv')
table4b = pd.read_csv('https://raw.githubusercontent.com/soltaniehha/Intro-to-Data-Analytics/main/data/table4b.csv')
table5  = pd.read_csv('https://raw.githubusercontent.com/soltaniehha/Intro-to-Data-Analytics/main/data/table5.csv')

In [2]:
table1

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


In [3]:
table2

,country,year,type,count
0,Afghanistan,1999,cases,745
1,Afghanistan,1999,population,19987071
2,Afghanistan,2000,cases,2666
3,Afghanistan,2000,population,20595360
4,Brazil,1999,cases,37737
5,Brazil,1999,population,172006362
6,Brazil,2000,cases,80488
7,Brazil,2000,population,174504898
8,China,1999,cases,212258
9,China,1999,population,1272915272


In [4]:
table3

,country,year,rate
0,Afghanistan,1999,745/19987071
1,Afghanistan,2000,2666/20595360
2,Brazil,1999,37737/172006362
3,Brazil,2000,80488/174504898
4,China,1999,212258/1272915272
5,China,2000,213766/1280428583


Spread across two data frames:

In [5]:
table4a

,country,1999,2000
0,Afghanistan,745,2666
1,Brazil,37737,80488
2,China,212258,213766


In [6]:
table4b

,country,1999,2000
0,Afghanistan,19987071,20595360
1,Brazil,172006362,174504898
2,China,1272915272,1280428583


These are all representations of the same underlying information, but they are not equally easy to use. One dataset, the tidy dataset, will be much easier to work with inside the pandas framework and many other libraries.

There are three interrelated rules which make a dataset tidy:

1. Each variable must have its own column.
2. Each observation must have its own row.
3. Each value must have its own cell.

Figure below shows the rules visually

<img src="https://github.com/soltaniehha/Intro-to-Data-Analytics/blob/main/figs/tidy-1.png?raw=true" width="1000" align="center"/>

*Image from [R for Data Science](https://r4ds.had.co.nz/)

In this example, only `table1` is tidy. It's the only representation where each column is a variable.

It's advantageous to have a consistent definition of *tidy* data. This allows to learn the tools that work with that structure. Another big benefit of having variables in columns and observations in rows is that it allows Pandas' vectorized nature to stand out; this includes both the vectorized functionalities within Pandas and Numnpy as well as much higher performance while performing computation within columns.

**Pandas**, **Scikit-learn**, **Matplotlib**/**Seaborn** and many other packages are designed to work with tidy data.



## Spreading and Gathering
The principles of tidy data seem so obvious that you might wonder if you'll ever encounter a dataset that isn't tidy. Unfortunately, however, most data that you will encounter will be untidy. There are two main reasons:

Most people aren't familiar with the principles of tidy data, and it's hard to derive them yourself unless you spend a lot of time working with data.

Data is often organized to facilitate some use other than analysis. For example, data is often organized to make data entry as easy as possible.

This means for most real analyses, you'll need to do some tidying. The first step is always to figure out what the variables and observations are. Sometimes this is easy; other times you'll need to consult with the people who originally generated the data. The second step is to resolve one of two common problems:

1. One variable might be spread across multiple columns.
2. One observation might be scattered across multiple rows.

Typically a dataset will only suffer from one of these problems; it'll only suffer from both if you're really unlucky! To fix these problems, you'll need one of the two strategies: spreading or gathering.


### Gathering
A common problem is a dataset where some of the column names are not names of variables, but values of a variable. Take `table4a`: the column names `1999` and `2000` represent values of the `year` variable, and each row represents two observations, not one.


In [7]:
table4a

,country,1999,2000
0,Afghanistan,745,2666
1,Brazil,37737,80488
2,China,212258,213766


To tidy a dataset like this, we need to "gather" those columns into a new pair of variables. To describe that operation we need three parameters:

* The set of columns that represent values to be gathered, `value_vars`. In this example, those are the columns `1999` and `2000`.
* The name of the variable whose values form the column names. That is called `var_name`, and here it is `year`.
* The name of the variable whose values are spread over the cells. That is called `value_name`, and here it's the number of `cases`.
* And finally, a list of columns that we want to keep without gathering them, `id_vars`. Here it is `country`.

Together those parameters generate the call to Pandas' `melt` function:

In [8]:
table4a_tidy = table4a.melt(id_vars=['country'], value_vars=['1999', '2000'], var_name='year', value_name='cases')
table4a_tidy

,country,year,cases
0,Afghanistan,1999,745
1,Brazil,1999,37737
2,China,1999,212258
3,Afghanistan,2000,2666
4,Brazil,2000,80488
5,China,2000,213766


* `id_vars`: Columns to keep (not gathered)
* `value_vars`: Columns to gather
* `var_name`: Name of new gathered column
* `value_name`: Name of new value column

In the final result, the gathered columns are dropped, and we get new `var_name` and `value_name` columns. Otherwise, the relationships between the original variables are preserved. Visually, this is shown in figure below 

<img src="https://github.com/soltaniehha/Intro-to-Data-Analytics/blob/main/figs/gather-table4a.png?raw=true" width="800" align="center"/>

*Image from [R for Data Science](https://r4ds.had.co.nz/)

We can use `melt()` to tidy `table4b` in a similar fashion. The only difference is the variable stored in the cell values:

In [9]:
table4b_tidy = table4b.melt(id_vars=['country'], value_vars=['1999', '2000'], var_name='year', value_name='population')
table4b_tidy

,country,year,population
0,Afghanistan,1999,19987071
1,Brazil,1999,172006362
2,China,1999,1272915272
3,Afghanistan,2000,20595360
4,Brazil,2000,174504898
5,China,2000,1280428583


We now combine the new `table4a` and `table4b` into a single dataframe. We will see the details of joining in the future notebooks.


In [10]:
table4a_tidy.merge(table4b_tidy, on=['country', 'year'])

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Brazil,1999,37737,172006362
2,China,1999,212258,1272915272
3,Afghanistan,2000,2666,20595360
4,Brazil,2000,80488,174504898
5,China,2000,213766,1280428583


### Spreading
Spreading is the opposite of gathering. You use it when an observation is scattered across multiple rows. For example, take `table2`: an observation is a country in a year, but each observation is spread across two rows:


In [11]:
table2

,country,year,type,count
0,Afghanistan,1999,cases,745
1,Afghanistan,1999,population,19987071
2,Afghanistan,2000,cases,2666
3,Afghanistan,2000,population,20595360
4,Brazil,1999,cases,37737
5,Brazil,1999,population,172006362
6,Brazil,2000,cases,80488
7,Brazil,2000,population,174504898
8,China,1999,cases,212258
9,China,1999,population,1272915272


To tidy this up, we first analyze the representation in similar way to `melt()`. This time, however, we only need three parameters:

* The column that contains variable names, the `columns` column. Here, it's `type`.
* The column that contains values from multiple variables, the `values` column. Here it's `count`.
* And `index` will be used to keep the variables we don't want to spread.

Once we've figured that out we can use the `pivot` function:

In [12]:
table2.pivot(index=['country', 'year'], columns='type',values='count')

type               cases  population
country     year                    
Afghanistan 1999     745    19987071
            2000    2666    20595360
Brazil      1999   37737   172006362
            2000   80488   174504898
China       1999  212258  1272915272
            2000  213766  1280428583

Note that the `pivot()` function is not purely designed for this task, however, it's perfectly capable of handling it. If we want to remove the variables from index we can use `reset_index` and `rename_axis` as the following code shows:

In [13]:
table2_tidy = table2.pivot(index=['country', 'year'], columns='type',values='count').reset_index().rename_axis(None, axis=1)
table2_tidy

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


<img src="https://github.com/soltaniehha/Intro-to-Data-Analytics/blob/main/figs/spread-table2.png?raw=true" width="800" align="center"/>

*Image from [R for Data Science](https://r4ds.had.co.nz/)

## Splitting and uniting
So far you've learned how to tidy `table2` and `table4`, but not `table3`. `table3` has a different problem: we have one column (`rate`) that contains two variables (`cases` and `population`). To fix this problem, we'll need the `str.split()` function. Conversly, we sometimes have to combine multiple columns into one.

### Split
`str.split()` pulls apart one column into multiple columns, by splitting wherever a separator character appears. Take `table3`:


In [14]:
table3_tidy = table3.copy()
table3_tidy[['cases','population']] = table3_tidy['rate'].str.split('/', expand=True)
table3_tidy.drop('rate', axis=1, inplace=True)
table3_tidy

,country,year,cases,population
0,Afghanistan,1999,745,19987071
1,Afghanistan,2000,2666,20595360
2,Brazil,1999,37737,172006362
3,Brazil,2000,80488,174504898
4,China,1999,212258,1272915272
5,China,2000,213766,1280428583


<img src="https://github.com/soltaniehha/Intro-to-Data-Analytics/blob/main/figs/separate.png?raw=true" width="800" align="center"/>

*Image from [R for Data Science](https://r4ds.had.co.nz/)

By default the new columns will have a character type. We can cast the appropriate type:

In [15]:
table3_tidy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   country     6 non-null      object
 1   year        6 non-null      int64 
 2   cases       6 non-null      object
 3   population  6 non-null      object
dtypes: int64(1), object(3)
memory usage: 320.0+ bytes


In [16]:
table3_tidy["cases"] = pd.to_numeric(table3_tidy["cases"])
table3_tidy["population"] = pd.to_numeric(table3_tidy["population"])
table3_tidy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   country     6 non-null      object
 1   year        6 non-null      int64 
 2   cases       6 non-null      int64 
 3   population  6 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 320.0+ bytes


### Unite
In this example with `table5` we can combines multiple columns into a single column. You'll need it much less frequently than `split()`, but it's still a useful tool to have in your back pocket.

Consider `table5`:

In [17]:
table5

,country,century,year,rate
0,Afghanistan,19,99,745/19987071
1,Afghanistan,20,0,2666/20595360
2,Brazil,19,99,37737/172006362
3,Brazil,20,0,80488/174504898
4,China,19,99,212258/1272915272
5,China,20,0,213766/1280428583


In [18]:
table5_tidy = table5.copy()
table5_tidy['year'] = pd.to_numeric(table5_tidy['century'].astype(str) + table5_tidy['year'].astype(str).str.zfill(2))
table5_tidy.drop(['century'], axis=1, inplace=True)
table5_tidy

,country,year,rate
0,Afghanistan,1999,745/19987071
1,Afghanistan,2000,2666/20595360
2,Brazil,1999,37737/172006362
3,Brazil,2000,80488/174504898
4,China,1999,212258/1272915272
5,China,2000,213766/1280428583


<img src="https://github.com/soltaniehha/Intro-to-Data-Analytics/blob/main/figs/unite.png?raw=true" width="800" align="center"/>

*Image from [R for Data Science](https://r4ds.had.co.nz/)